In [35]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain, LLMChain
import pandas as pd
import os

load_dotenv()

True

### Calling LLM and Calling DataFrame

In [36]:
llm = ChatGoogleGenerativeAI(model="gemini-pro",convert_system_message_to_human = True,temperature=0)
moods_data = pd.read_csv("moods_data.csv")
relative_moods = list(moods_data['Moods'])

### Fetch Data from Weather API

In [37]:
import requests
import json

# Accessing IP-based location information
ip_url = "https://api.ipify.org?format=json"
response = requests.get(ip_url)
ip_data = json.loads(response.text)
ip_address = ip_data["ip"]

# Using a free weather API 
weather_api_key = os.environ['weather_api_key']  
weather_api_base_url = "http://api.openweathermap.org/data/2.5/weather"

# Retrieving location information based on IP address
geolocation_url = f"https://freegeoip.app/json/{ip_address}"
geolocation_response = requests.get(geolocation_url)
geolocation_data = json.loads(geolocation_response.text)
city = geolocation_data.get("city")

# Making the weather API request
weather_url = f"{weather_api_base_url}?q={city}&appid={weather_api_key}"
weather_response = requests.get(weather_url)
weather_data = json.loads(weather_response.text)

# Extracting relevant weather information
if weather_data.get("cod") == 200:  # Check for successful response
    weather_description = weather_data["weather"][0]["description"]
    temperature_kelvin = weather_data["main"]["temp"]
    feels_like = round(weather_data["main"]["feels_like"] - 273.15, 2)
    temperature_celsius = temperature_kelvin - 273.15  # Convert to Celsius
    weather_details = f"It's {city} and here weather condition is {weather_description} and temperature is {temperature_celsius:.2f}°C"
    print(weather_details)
else:
    print("Error: Unable to retrieve weather data.")


It's Kolkata and here weather condition is mist and temperature is 11.97°C


### Method-1  -- Using SequentialChain

In [64]:
def song_mood_analyzer(weather_details,relative_moods):
    template1 = """Can you suggest me specific song moods by the weather condition and temperature, 
            {weather_details}, I want only two song moods and not the song name, any kind of description is also not required, 
            return only headings one after another with a comma between them, output should be in a python list mood1,mood2 """
    prompt1 = PromptTemplate(input_variables=["weather_details"],template=template1)
    chain1 = LLMChain(llm=llm,prompt=prompt1,output_key="moods")

    template2 = """
                    find out the similar and strongly specific mood from the list of moods {relative_moods},
                    which are similar with the list {moods}, calculate the similarity score and also see on basis of similarity score, 
                    i want only mood headings only,  no bullet points, I want only similar moods in a pattern like mood1|mood2|mood3...
                """
    prompt2 = PromptTemplate(input_variables=["moods","relative_moods"],template=template2)
    chain2 = LLMChain(llm=llm,prompt=prompt2,output_key="founded_moods")

    final_chain = SequentialChain(
    chains=[chain1,chain2],
    input_variables=["weather_details","relative_moods"],
    output_variables=["moods","founded_moods"],
    verbose=True
    )

    result = final_chain({"weather_details":f"{weather_details}","relative_moods":f"{relative_moods}"})

    return result
    

In [66]:
founded_moods = song_mood_analyzer(weather_details,relative_moods)["founded_moods"]
founded_moods = founded_moods.split("|")
founded_moods



> Entering new SequentialChain chain...

> Finished chain.


['Calm',
 'Melancholy',
 'Beautiful',
 'Chill',
 'Easy Listening',
 'Lullaby',
 'Sentimental',
 'Smooth',
 'Soul',
 'Upbeat']

### Method-1  -- It's my Way 😂

In [26]:
prompt = f"""Can you suggest me specific song moods by the weather condition and temperature, 
            {weather_details}, I want only two song moods and not the song name, any kind of description is also not required, 
            return only headings one after another with a comma between them, output should be in format mood1,mood2 """

moods = llm.invoke(prompt).content.split(',')

similar_prompt = f"""
                    find out the similar specific mood from the list of moods {list(moods_data['Moods'])},
                    which are similar with the list {moods}, headings only, no bullet points
                """
founded_moods = llm.invoke(similar_prompt).content.split(', ')


In [67]:
updated_playlist = moods_data[moods_data["Moods"].isin(founded_moods)].reset_index(drop=True)

updated_playlist

,artist,song,link,text,Moods
0,Noa,Path To Follow,/n/noa/path+to+follow_10194425.html,I found a path to follow \r\nI knew it straig...,Soul
1,Yngwie Malmsteen,Don't Let It End,/y/yngwie+malmsteen/dont+let+it+end_20531862.html,You say the end of time is coming far too near...,Beautiful
2,Foo Fighters,Butterflies,/f/foo+fighters/butterflies_20054720.html,Where's this come from? he's just as \r\nWher...,Calm
3,Hank Williams Jr.,Fax Me A Beer,/h/hank+williams+jr/fax+me+a+beer_21033703.html,Well here we are smack in the middle of the 90...,Chill
4,Modern Talking,Don't Let Me Go,/m/modern+talking/dont+let+me+go_20094603.html,"A lonely avenue, it's raining in my heart \r\...",Easy Listening
5,Marianne Faithfull,Bored By Dreams,/m/marianne+faithfull/bored+by+dreams_20088568...,Things are never what they seem \r\nPlay a pa...,Lullaby
6,George Michael,Heartbeat,/g/george+michael/heartbeat_20059370.html,"Another summer, another vacation is over \r\n...",Sentimental
7,Conway Twitty,(I Can't Believe) She Gives It All To Me,/c/conway+twitty/i+cant+believe+she+gives+it+a...,I pinch myself when I wake up each mornin' \r...,Smooth
8,Kelly Clarkson,Catch My Breath,/k/kelly+clarkson/catch+my+breath_21048402.html,I don't wanna be left behind \r\nDistance was...,Soul
9,Conway Twitty,Halfway To Heaven,/c/conway+twitty/halfway+to+heaven_20213954.html,"We're halfway to heaven \r\nDon't leave me, n...",Upbeat
